# Final Project Example

## Retrieve/Scrape the Data

In [ ]:
import requests
import json
import pandas as pd

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAKIqKQEAAAAArvCY9UR1YF2ItjDVfvxRj%2BacYLI%3Dobk23ATK7g32XQ7CQiotckLl5EkTIfqJQGvIWWSNOUNRxTN0Dc'
headers = {'Authorization':('Bearer '+ bearer_token)}

# In this example, only those tweets with photos/images are stored

n = 10000                         # The total number of tweets we want
max_results = 100                 # The number of tweets to pull per request; must be between 10 and 100
total_retrieved = 0               # To keep track of when to stop
next_token = ""                   # Must be empty on first iteration
search_term = "covid%20gofundme"  # To form an advanced query, see here: https://twitter.com/search-advanced?lang=en
since_id = "1372000000000000000"  # The id of the oldest tweet you want to retrieve

# Create the empty DataFrame with the columns you want
df = pd.DataFrame(columns=['id', 'created_at', 'retweets', 'likes', 'replies', 'quotes', 'user_followers', 'user_following', 'user_listed', 'user_tweets', 'has_media', 'url', 'lang', 'text'])
df.set_index('id', inplace=True)

# stop when we have n results
while total_retrieved < n:

  # the first time through the loop, we do not need the next_token parameter
  if next_token == "":
    url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}&since_id={since_id}'
  else:
    url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}&since_id={since_id}&next_token={next_token}'

  # These are the extra parameters we will add to the querystring; we won't store them all though; just want you to see what's possible
  url += f'&user.fields=id,public_metrics'
  url += f'&tweet.fields=attachments,public_metrics,text,created_at,author_id,lang'
  url += f'&expansions=attachments.media_keys,author_id'
  url += f'&media.fields=media_key,type,url'

  # make the request to the Twitter API Recent Search endpoint
  response = requests.request("GET", url, headers=headers)
  try:  # Just in case we get an error
    json_data = json.loads(response.text)
  except:
    print(response.text)
  
  # Error checking; print the results if valid data is not retrieved
  if not 'data' in json_data:
    json_clean = json.dumps(json_data, indent=2, sort_keys=True)
    print(json_clean)


  for tweet in json_data['data']:
    media_key = ""  # Reset to empty each time through the loop so that we can use it for a condition later

    # Store the data into variables
    tweet_id = tweet['id']
    try:
      author_id = tweet['author_id']
    except:
      print(tweet)
    created_at = tweet['created_at']
    retweet_count = tweet['public_metrics']['retweet_count']
    like_count = tweet['public_metrics']['like_count']
    reply_count = tweet['public_metrics']['reply_count']
    quote_count = tweet['public_metrics']['quote_count']
    user_followers = ""
    user_following = ""
    user_listed = ""
    user_tweets = ""
    has_media = False
    image_url = ""
    lang = tweet['lang']
    text = tweet['text']

    # Find out if there is media
    if 'attachments' in tweet:
      if 'media_keys' in tweet['attachments']:
        media_key = tweet['attachments']['media_keys'][0]

    # If there is a media key in this tweet, iterate through tweet['includes']['media'] until we find it
    if media_key != "":
      for media in json_data['includes']['media']:
        if media['media_key'] == media_key: # Only if the media_key matches the one we stored
          has_media = True
          if media['type'] == 'photo':      # Only if it is a photo; ignore videos
            image_url = media['url']        # Store the url in a variable
            break
    
    
    # Iterate through all authors until we find the author of this tweet; then store their metrics
    for author in json_data['includes']['users']:
      if author['id'] == author_id:
        user_followers = author['public_metrics']['followers_count']
        user_following = author['public_metrics']['following_count']
        user_listed = author['public_metrics']['listed_count']
        user_tweets = author['public_metrics']['tweet_count']
        break
                
    # Only add the record in the DataFrame if a photo is found
    df.loc[tweet_id] = [created_at, retweet_count, like_count, reply_count, quote_count, user_followers, user_following, user_listed, user_tweets, has_media, image_url, lang, text]


  # keep track of where to start next time, but quit if there are no more results
  try:
    total_retrieved += json_data['meta']['result_count']
    next_token = json_data['meta']['next_token']
  except:
    break  

df.to_csv('twitter.csv')
df.head()

  
    
    
    
    
## Exploratory Data Analysis (i.e. Data Understanding Phase)
### Begin with univariate analyses

In [ ]:
import pandas as pd
df = pd.read_csv('twitter.csv')
df.shape

In [ ]:
# Convert these numbers to categories
df['id'] = df['id'].astype('object')

# Parse out the date into potentially useful features
df['created_at'] = pd.to_datetime(df['created_at'])
df['Weekday'] = df['created_at'].dt.day_name()
df['DayOfWeek'] = df['created_at'].dt.dayofweek
df['Hour'] = df['created_at'].dt.hour
df.drop(columns=['created_at'], inplace=True)

df.describe()

In [ ]:
df.skew()


  
### Continue with bivariate analyses

In [ ]:
# Create a heatmap over a correlation table

import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 9))

matrix = np.triu(df.corr())
sns.heatmap(df.corr(), annot=True, fmt='.2f', 
            vmin=-1, vmax=1, center=0, cmap= 'coolwarm', 
            mask=matrix, square=True);

In [ ]:
df.dtypes

In [ ]:
sns.set(color_codes=True)
sns.jointplot(x='likes', y='quotes', data=df);

In [ ]:
sns.barplot(data=df, x="has_media", y="retweets");

# Text Processing

In [ ]:
!pip install --upgrade pip
!pip install pyLDAvis
!pip install bokeh
!pip install gensim
!pip install spacy
!pip install logging
!pip install warnings
!pip install matplotlib
!pip install nltk
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install -U seaborn
!pip install translators --upgrade

## Translate Non-English Tweets to English

In [97]:
def translate(text):
  import translators as ts
  translated = ""

#   # input languages
#   try:
#     print(ts.deepl(text, from_language='auto', to_language='en'))
#   except:
#     pass

#   # output language_map
#   try:
#     print(ts._deepl.language_map)
#   except:
#     pass

  # professional field
  try:
    translated = ts.alibaba(text, professional_field='general') # ("general","message","offer")
  except:
    pass

  if translated == "":
    try:
      translated = ts.baidu(text, professional_field='common') # ('common','medicine','electronics','mechanics')
    except:
      pass

  # host service
  if translated == "":
    try:
      translated = ts.google(text, if_use_cn_host=True)
      translated = ts.bing(text, if_use_cn_host=False)
    except:
      pass

  return translated

print(translate("La gran Soledad Bravo cayó ví"))

{'pl': {'pl', 'it', 'fr', 'de', 'es', 'pt', 'nl', 'ja', 'zh', 'en', 'ru'}, 'it': {'pl', 'it', 'fr', 'de', 'es', 'pt', 'nl', 'ja', 'zh', 'en', 'ru'}, 'fr': {'pl', 'it', 'fr', 'de', 'es', 'pt', 'nl', 'ja', 'zh', 'en', 'ru'}, 'de': {'pl', 'it', 'fr', 'de', 'es', 'pt', 'nl', 'ja', 'zh', 'en', 'ru'}, 'es': {'pl', 'it', 'fr', 'de', 'es', 'pt', 'nl', 'ja', 'zh', 'en', 'ru'}, 'pt': {'pl', 'it', 'fr', 'de', 'es', 'pt', 'nl', 'ja', 'zh', 'en', 'ru'}, 'nl': {'pl', 'it', 'fr', 'de', 'es', 'pt', 'nl', 'ja', 'zh', 'en', 'ru'}, 'ja': {'pl', 'it', 'fr', 'de', 'es', 'pt', 'nl', 'ja', 'zh', 'en', 'ru'}, 'zh': {'pl', 'it', 'fr', 'de', 'es', 'pt', 'nl', 'ja', 'zh', 'en', 'ru'}, 'en': {'pl', 'it', 'fr', 'de', 'es', 'pt', 'nl', 'ja', 'zh', 'en', 'ru'}, 'ru': {'pl', 'it', 'fr', 'de', 'es', 'pt', 'nl', 'ja', 'zh', 'en', 'ru'}}
The great loneliness Bravo fell ví


# Modeling

In [52]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor # Import Decision Tree Regression algorithm
from sklearn.ensemble import GradientBoostingRegressor # Import XGBoost algorithm 
from sklearn.model_selection import train_test_split # Import train_test_split function
# for a completelist of available algorithms: https://scikit-learn.org/stable/supervised_learning.html
# Which one should I use?: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

In [53]:
# Create dummy codes for all features and not the label

df.drop(columns=['id', 'text'], inplace=True)

for col in df.columns:
  if not pd.api.types.is_numeric_dtype(df[col]):
    df = pd.get_dummies(df, columns=[col], prefix=col)

df.head()

,retweets,likes,replies,quotes,user_followers,user_following,user_listed,user_tweets,has_media,id_1372922770024230912,...,text_🎈Shoutout to Cess M. for donating $100.\n\nYou’ve earned a Swagmate Reward: custom t-shirt &amp; hoodie https://t.co/H2b6TN2vY2\n\n*We’ll start shipping gifts in Q3 2021. \n\nThank you. \n\n#ad #giveaway,text_🎈Shoutout to Cess M. for donating $100.\n\nYou’ve earned a Swagmate Reward: custom t-shirt &amp; hoodie https://t.co/TxBC6iYCRv\n\n*We’ll start shipping gifts in Q3 2021. \n\nThank you.,text_🎈Shoutout to Cess M. for donating $100.\n\nYou’ve earned a Swagmate Reward: custom t-shirt &amp; hoodie https://t.co/z2ZgDHrben\n\n*We’ll start shipping gifts in Q3 2021. \n\nThank you. #ad,"text_👉Millions of people have still recieved no financial support at all - could you survive of no income, no benefits for 12 months? 👈\n\nPlease retweet ♻️ and Like ❤ and support those who have been #ExcludedUk #FirstInLastOut \n\nhttps://t.co/kxD1fGX0fy",text_💰💰It cost at least $350 an hour to keep the ranch open. During #COVID19 we have been struggling to care for our horses 🐎 and need your support. Donate today on #GoFundMe \nhttps://t.co/EYlcAtn8pH\n#InThisTogether https://t.co/f90nY9cupq,text_💰💰It cost at least $350 an hour to keep the ranch open. During #COVID19 we have been struggling to care for our horses 🐎 and need your support. Donate today on #GoFundMe \nhttps://t.co/EYlcAtn8pH\n#InThisTogether https://t.co/z9Laa5Stro,text_📢📢📢📢📢📢📢📢📢📢📢\n\n#SERVICIOPÚBLICO\n#AYUDAADIFUNDIR\n\nhttps://t.co/u0F8p9hPa0,"text_🔴 ¡Ayudemos! 🔴 La psiquiatra Lourdes Torrealba, la concejal Liris Loreto y Soledad Bravo piden colaboración para superar crisis de COVID-19 en GoFundMe\n\n👇🏻",text_🙏🏻🙏🏻🙏🏻\n\nAyuda a Pedro a superar el COVID-19 https://t.co/WE0pkupSvC,text_🚨🚨🚨\n\nPlease donate to Nzingha so she can afford to register for classes and complete her undergrad! Due to COVID and financial constraints she has to pay off a huge balance in her school fees. This is extremely urg*nt her last donation was 5 days ago!\n\nhttps://t.co/EzJvOxO7dp https://t.co/HoX2WYiOaI
0,818,0,0,0,983,1272,19,163569,0,0,...,0,0,0,0,0,0,0,0,0,0
1,60,0,0,0,12,25,0,2092,0,0,...,0,0,0,0,0,0,0,0,0,0
2,38,0,0,0,64,216,1,2380,0,0,...,0,0,0,0,0,0,0,0,0,0
3,346,0,0,0,351,501,13,17258,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,163,384,0,3128,1,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Split dataset in features and target variable

y = df.charges # Label
X = df.drop(columns=['charges']) # Features
X = X.select_dtypes(np.number)
X.head()

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,1,0,0,1,0,0,0,1
1,18,33.770,1,0,1,1,0,0,0,1,0
2,28,33.000,3,0,1,1,0,0,0,1,0
3,33,22.705,0,0,1,1,0,0,1,0,0
4,32,28.880,0,0,1,1,0,0,1,0,0


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test
X_train.head()

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
744,50,26.41,0,0,1,1,0,0,1,0,0
363,21,26.40,1,1,0,1,0,0,0,0,1
10,25,26.22,0,0,1,1,0,1,0,0,0
970,50,28.16,3,1,0,1,0,0,0,1,0
634,51,39.70,1,0,1,1,0,0,0,0,1


In [18]:
# Create Decision Tree regressor object
clf = DecisionTreeRegressor()

# Train Decision Tree regressor
clf = clf.fit(X_train,y_train)

# Predict the labels for test dataset
y_pred = clf.predict(X_test)

In [19]:
output_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred,})
output_df.head(10)

,Actual,Predicted
559,1646.42970,1263.24900
1087,11353.22760,11945.13270
1020,8798.59300,8457.81800
460,10381.47870,10702.64240
802,2103.08000,1964.78000
298,38746.35510,39983.42595
481,9304.70190,9048.02730
616,11658.11505,11658.37915
763,3070.80870,3558.62025
750,19539.24300,20149.32290


In [20]:
# Import scikit-learn metrics module. See complete list of Classification metrics here: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
from sklearn import metrics
    
print(f'R squared:\t{metrics.r2_score(y_test, y_pred)}')
print(f'MAE:\t\t{metrics.mean_absolute_error(y_test, y_pred)}')
print(f'RMSE:\t\t{metrics.mean_squared_error(y_test, y_pred)**(1/2)}')

R squared:	0.7192346766990366
MAE:		2964.1516434800997
RMSE:		6307.862354958355


In [21]:
# Create XGBoost regressor object
clr = GradientBoostingRegressor()

# Train Decision Tree regression
clr = clf.fit(X_train,y_train)

# Predict the labels for test dataset
y_pred = clr.predict(X_test)

In [22]:
output_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred,})
output_df.head(10)

,Actual,Predicted
559,1646.42970,1263.24900
1087,11353.22760,11842.62375
1020,8798.59300,8457.81800
460,10381.47870,10702.64240
802,2103.08000,1964.78000
298,38746.35510,55135.40209
481,9304.70190,9048.02730
616,11658.11505,11658.37915
763,3070.80870,3558.62025
750,19539.24300,20149.32290


In [23]:
from sklearn import metrics
    
print(f'R squared:\t{metrics.r2_score(y_test, y_pred)}')
print(f'MAE:\t\t{metrics.mean_absolute_error(y_test, y_pred)}')
print(f'RMSE:\t\t{metrics.mean_squared_error(y_test, y_pred)**(1/2)}')

R squared:	0.7184834368316932
MAE:		2987.658830445274
RMSE:		6316.295647265444


In [24]:
import pickle

# Save the model with the highest fit metric
pickle.dump(clr, open('stored_model.sav', 'wb'))  # OPTION 1: pickle

['stored_model.pkl']

In [25]:
# ...some time later

import pickle
import numpy as np

# OPTION 1: Using pickle
# load the model from 'stored_model.sav'
loaded_model = pickle.load(open('stored_model.sav', 'rb'))
print(type(loaded_model))

# for a single prediction, enter a row of data and reshape into numpy array
case = [0.543478, 0.245359, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0]
print(f'Single prediction {case}: {loaded_model.predict(np.array(case).reshape(1, -1))[0]}\n')

# for a batch prediction, enter a Pandas DataFrame or a Numpy array of arrays
predictions = loaded_model.predict(X_test) 
batch_results = pd.DataFrame({'Actual':y_test, 'Predicted':predictions, 'Diff':(predictions - y_test)})
print(f'MAE:\t{batch_results.Diff.abs().mean()}\n')
batch_results.head(5)

<class 'sklearn.tree._classes.DecisionTreeRegressor'>
Single prediction [0.543478, 0.245359, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0]: 16420.49455

MAE:	2987.6588304452744



,Actual,Predicted,Diff
559,1646.4297,1263.24900,-383.18070
1087,11353.2276,11842.62375,489.39615
1020,8798.5930,8457.81800,-340.77500
460,10381.4787,10702.64240,321.16370
802,2103.0800,1964.78000,-138.30000
